# Autenticação
Primeiro precisamos autenticar nossa sessão do Colab no Google e definimos o ID do projeto

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
#Informe o id do projeto nesta linha:

project_id = "curso-gcp-ml-tensorflow" #@param {type:"string"}


# Client Big Query

Como vamos buscar dados no bigquery, é necessário criar um cliente BG

In [0]:
# Call BigQuery and examine in dataframe
from google.cloud import bigquery

bgclient = bigquery.Client(project=project_id)

Demais imports:

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns, numpy as np
import shutil
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.15.0


#Copia os dados de treinamento e teste (CSV) para diretório local

In [0]:
%pwd

'/content'

In [0]:
!gsutil cp gs://curso-gcp-ml-tensorflow-ds/*.csv /content

Copying gs://curso-gcp-ml-tensorflow-ds/teste.csv...
Copying gs://curso-gcp-ml-tensorflow-ds/treino.csv...
/ [2 files][  1.9 MiB/  1.9 MiB]                                                
Operation completed over 2 objects/1.9 MiB.                                      


In [0]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   8293 teste.csv
  24955 treino.csv
  33248 total
==> teste.csv <==
3.3719999971722725,True,28,False,1,38.0,7146494315947640619
4.479999996243114,True,31,True,1,41.0,7146494315947640619
2.947999997527835,False,30,False,1,37.0,7146494315947640619
3.0899999974087553,True,19,False,1,39.0,7146494315947640619
3.203999997313156,False,26,True,1,36.0,7146494315947640619
3.855999996766395,True,25,True,1,38.0,7146494315947640619
0.9919999991681182,True,30,True,1,28.0,7146494315947640619
2.777999997670395,False,20,False,1,36.0,7146494315947640619
3.4299999971236343,True,29,True,1,40.0,7146494315947640619
3.259999997266195,True,38,True,1,36.0,7146494315947640619

==> treino.csv <==
2.551999997859917,False,18,True,1,38.0,7146494315947640619
4.166999996505593,True,19,False,1,37.0,7146494315947640619
2.551999997859917,True,21,False,1,35.0,7146494315947640619
3.4299999971236343,False,33,True,1,37.0,7146494315947640619
2.0129999983119173,False,16,False,1,35.0,7146494315947640619
3.117999997385275,True

# Cria o modelo

Determina o que será lido do CSV! (pois o mesmo está sem header)

In [0]:
CSV_COLUMNS = ['weight_kilos', 'is_male', 'mother_age', 'mother_married', 'plurality', 'gestation_weeks', 'hashmonth']
DEFAULTS =    [[0.0],          ['null'],  [0.0],        ['null'],         ['null'],    [0.0],             ['nokey']  ]
LABEL_COLUMN = 'weight_kilos'
KEY_COLUMN = 'hashmonth'

TRAIN_STEPS = 1000

Uma função inteligente, retorna a função de leitura como se fosse um lambda, de acordo com o modo (TRAIN ou !TRAIN)

In [0]:
# Define a função que lê datasets. Observe que é uma função
def read_dataset(filename, mode, batch_size=512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults=DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = (tf.data.TextLineDataset(file_list)  # Read text file
                 .map(decode_csv))  # Transform each elem by applying decode_csv fn
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size=10*batch_size)
    else:
        num_epochs = 1 # end-of-input after this
 
    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset
  return _input_fn

Define as colunas

In [0]:
colunas_input_layer = [
    tf.feature_column.indicator_column( tf.feature_column.categorical_column_with_vocabulary_list('is_male', ['True', 'False', 'Unknown'])),
    tf.feature_column.numeric_column('mother_age'),
    tf.feature_column.indicator_column( tf.feature_column.categorical_column_with_vocabulary_list('mother_married', ['True', 'False', 'Unknown'])),
    tf.feature_column.indicator_column( tf.feature_column.categorical_column_with_vocabulary_list('plurality', ['1', '2', '3', '4', '5', '6'])),
    tf.feature_column.numeric_column('gestation_weeks')
]

In [0]:
# Cria função de camada de entrada para predição (a ser usada na avaliação)
def serving_input_fn():
    feature_placeholders = {
        'is_male': tf.placeholder(tf.string, [None]),
        'mother_age': tf.placeholder(tf.float32, [None]),
        'mother_married': tf.placeholder(tf.string, [None]),
        'plurality': tf.placeholder(tf.string, [None]),
        'gestation_weeks': tf.placeholder(tf.float32, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [0]:
EVAL_INTERVAL = 300
run_config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL, keep_checkpoint_max = 3)

In [0]:
estimator = tf.estimator.DNNRegressor(
                     model_dir = 'modelo_treinado',
                     feature_columns = colunas_input_layer,
                     hidden_units = [64, 32],
                     config = run_config)

INFO:tensorflow:Using config: {'_model_dir': 'modelo_treinado', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb3c3d042b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Cria configurações de treino e de teste

In [0]:
configuracoes_de_treino = tf.estimator.TrainSpec(
    input_fn = read_dataset('treino.csv', mode = tf.estimator.ModeKeys.TRAIN),
    max_steps = TRAIN_STEPS)

In [0]:
exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
configuracoes_de_avaliacao = tf.estimator.EvalSpec(
    input_fn = read_dataset('teste.csv', mode = tf.estimator.ModeKeys.EVAL),
    steps = None,
    start_delay_secs = 60, # start evaluating after N seconds
    throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
    exporters = exporter)

In [0]:
shutil.rmtree('modelo_treinado', ignore_errors = True) # start fresh each time

In [0]:
tf.estimator.train_and_evaluate(estimator=estimator,
                                train_spec= configuracoes_de_treino,
                                eval_spec=configuracoes_de_avaliacao)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Skipping training since max_steps has already saved.


(None, None)

In [0]:
!ls -R modelo_treinado/export/exporter

modelo_treinado/export/exporter:
1583285961

modelo_treinado/export/exporter/1583285961:
saved_model.pb	variables

modelo_treinado/export/exporter/1583285961/variables:
variables.data-00000-of-00002  variables.data-00001-of-00002  variables.index


Apenas para teste: Faz e exibe predições

In [0]:
prediction = estimator.predict(input_fn = read_dataset('teste.csv', mode = tf.estimator.ModeKeys.PREDICT))
for i, p in enumerate(prediction):
    print(i, p)

Streaming output truncated to the last 5000 lines.
3294 {'predictions': array([3.5559194], dtype=float32)}
3295 {'predictions': array([3.229622], dtype=float32)}
3296 {'predictions': array([3.432129], dtype=float32)}
3297 {'predictions': array([3.0253341], dtype=float32)}
3298 {'predictions': array([3.4098215], dtype=float32)}
3299 {'predictions': array([2.2217968], dtype=float32)}
3300 {'predictions': array([3.4952269], dtype=float32)}
3301 {'predictions': array([3.46898], dtype=float32)}
3302 {'predictions': array([3.285899], dtype=float32)}
3303 {'predictions': array([3.0802572], dtype=float32)}
3304 {'predictions': array([3.438598], dtype=float32)}
3305 {'predictions': array([3.4383838], dtype=float32)}
3306 {'predictions': array([3.2634926], dtype=float32)}
3307 {'predictions': array([3.3598003], dtype=float32)}
3308 {'predictions': array([3.3773592], dtype=float32)}
3309 {'predictions': array([3.2690425], dtype=float32)}
3310 {'predictions': array([3.3797], dtype=float32)}
3311 {

In [0]:
# Copiar o export do modelo para o Storage
#!gsutil cp -r ./modelo_treinado gs://curso-gcp-ml-tensorflow-ds/

# para criar um modelo com treino e teste separados!

In [0]:
shutil.rmtree('modelo_treinado', ignore_errors = True) # start fresh each time

In [0]:
estimator.train(
    input_fn = read_dataset('treino.csv', mode = tf.estimator.ModeKeys.TRAIN),
    max_steps = TRAIN_STEPS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into modelo_treinado/model.ckpt.
INFO:tensorflow:loss = 16991.648, step = 1
INFO:tensorflow:global_step/sec: 33.4566
INFO:tensorflow:loss = 134.66255, step = 101 (2.990 sec)
INFO:tensorflow:global_step/sec: 35.2457
INFO:tensorflow:loss = 120.95633, step = 201 (2.837 sec)
INFO:tensorflow:global_step/sec: 35.0797
INFO:tensorflow:loss = 127.82497, step = 301 (2.856 sec)
INFO:tensorflow:global_step/sec: 35.6275
INFO:tensorflow:loss = 122.63486, step = 401 (2.805 sec)
INFO:tensorflow:global_step/sec: 35.9601
INFO:tensorflow:loss = 121.28214, step = 501 (2.778 sec)
INFO:tensorflow:global_step/sec: 35.8359
INFO:tensorflow:loss = 129.06587, step = 601 (2.790 sec)
INFO:tensorflow:global_step/sec: 35.4185
INFO:tensorflow:l

In [0]:
estimator.evaluate(input_fn = read_dataset('teste.csv', mode = tf.estimator.ModeKeys.EVAL),
    steps = None)

In [0]:
!ls modelo_treinado